Create Point to Site Virtual Private Network (VPN)
==================================
As part of the migration process, if you do not already have an Azure VPN and Gateway setup, this notebook will help you get one created and verify you have the connectivity necessary to securely host your new Azure SQL Server Virtual Machines. 

If a S2S connection is needed instead of a P2S, then see [Create and manage S2S VPN connections using PowerShell](https://docs.microsoft.com/en-us/azure/vpn-gateway/vpn-gateway-tutorial-vpnconnection-powershell) for more information. S2S may be out of scope for a data user and can be attempted by a network administrator.



There are a handful of steps you have to walk through to get your resource group and VPN configured.

Steps of this procedure include:
1. Connect to Azure subscription
1. Provision resource group for VPN
2. Create VPN
3. Provision IP address for Gateway
4. Create Gateway

<b>NOTE: Fill all the below variables with names in order to proceed ahead. </b>

In [ ]:
$RG1         = ""                     # Resource group name(It should be of alphabets case insensitive)
$VNet1       = ""                     # VPN name of your choice(It should be of alphabets case insensitive)
$Location1   = ""                     # Specify the valid location such as 'West US 2','EASTUS' etc...
$FESubnet1   = ""                     # Frontend subnet name of your choice(It should be of alphabets case insensitive)
$VNet1Prefix = ""                     # VNet default I.P format would be like this '10.1.0.0/16'
$FEPrefix1   = ""                     # Frontend Gateway default IP format would be '10.1.0.0/24'
$GwPrefix1   = ""                     # Default value for Gateway subnet would be '10.1.255.0/27'
$Gw1         = ""                     # Gateway name of your choice (It should be of alphabets case insensitive)
$GwIP1       = ""                     # Gateway IP name of your choice(It should be of alphabets case insensitive)
$GwIPConf1   = ""                     # Gateway IP Config name of your choice(It would be alphanumeric case insensitive)

## <b>Connect to Azure Account</b>

In [ ]:
Connect-AzAccount

## <b>Get Subscription </b>
Below command will open a Dialouge Box with list of subscriptions.
Selecting one of those will set that subscription for rest of the commands.

In [ ]:
$subscription = Get-AzSubscription | Out-GridView -PassThru
Set-AzContext -SubscriptionName $subscription

## <b>Create the resource group </b>
Create a resource group with the _New-AzResourceGroup_ command. An Azure resource group is a logical container into which Azure resources are deployed and managed. A resource group must be created first.

In [ ]:
# Create Azure resource group, if necessary
$RG1 = Get-AzResourceGroup -Name $RG1

if (!$RG1)
{
    # Need to create a new resource group
    Write-Output "Resource Group $RG1 does not exist. Creating..."
    $RG1 = New-AzResourceGroup -Name $RG1 -Location $Location1
}

## <b>Create a VPN</b>
Azure gateway provides cross-premises connectivity and P2S VPN server functionality for your VPN. Add a gateway to an existing VPN or create a new VPN and the gateway. Notice that the example specifies the name of the gateway subnet specifically. You must always specify the name of the gateway subnet as "GatewaySubnet" in order for it to function properly. This example creates a new VPN with two subnets: Frontend and GatewaySubnet using New-AzVirtualNetworkSubnetConfig and New-AzVirtualNetwork:

In [ ]:
$fesub1 = New-AzVirtualNetworkSubnetConfig -Name $FESubnet1 -AddressPrefix $FEPrefix1
$gwsub1 = New-AzVirtualNetworkSubnetConfig -Name 'GatewaySubnet' -AddressPrefix $GwPrefix1
$vnet   = New-AzVirtualNetwork `
            -Name $VNet1 `
            -ResourceGroupName $RG1 `
            -Location $Location1 `
            -AddressPrefix $VNet1Prefix `
            -Subnet $fesub1,$gwsub1

## <b>Request a public IP address for the gateway</b>
Azure VPN gateways communicate with your on-premises VPN devices over the Internet to performs IKE (Internet Key Exchange) negotiation and establish IPsec tunnels. Create and assign a public IP address to your gateway as shown in the example below with New-AzPublicIpAddress and New-AzVirtualNetworkGatewayIpConfig:

 #### Important :
 Currently, you can only use a Dynamic public IP address for the gateway. Static IP address is not supported on Azure VPN gateways.

In [ ]:
$gwpip    = New-AzPublicIpAddress -Name $GwIP1 -ResourceGroupName $RG1 `
              -Location $Location1 -AllocationMethod Dynamic
$subnet   = Get-AzVirtualNetworkSubnetConfig -Name 'GatewaySubnet' `
              -VirtualNetwork $vnet
$gwipconf = New-AzVirtualNetworkGatewayIpConfig -Name $GwIPConf1 `
              -Subnet $subnet -PublicIpAddress $gwpip

## <b>Create a gateway </b>

A gateway can take 45 minutes or more to create. Once the gateway creation has completed, you can create a connection between your VPN and another VNet. Or create a connection between your VPN and an on-premises location. Create a gateway using the New-AzVirtualNetworkGateway cmdlet.

In [ ]:
New-AzVirtualNetworkGateway -Name $Gw1 -ResourceGroupName $RG1 `
  -Location $Location1 -IpConfigurations $gwipconf -GatewayType Vpn `
  -VpnType RouteBased -GatewaySku VpnGw1